# 셀레니움으로 금감원 비디오자료 자동 다운로드 구현

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import requests, os

# 다운로드 받을 폴더 생성
DOWNLOAD_DIR = "videos"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


# ✅ Chrome 드라이버 초기화 함수 정의
def init_driver():
    service = Service(ChromeDriverManager().install())  # 최신 크롬 드라이버 자동 설치
    opts = webdriver.ChromeOptions()
    opts.add_argument("--headless")  # 브라우저 창 띄우지 않고 백그라운드 실행
    return webdriver.Chrome(service=service, options=opts)


driver = init_driver()  # 최초 드라이버 실행

# ✅ 1~12페이지까지 반복
for page in range(1, 13):
    try:
        video_page_urls = []  # 현재 페이지에 있는 동영상 게시물 링크들 저장

        # 페이지 URL 구성 후 이동
        url = f"https://www.fss.or.kr/fss/bbs/B0000203/list.do?menuNo=200686&pageIndex={page}"
        driver.get(url)

        # 게시물 썸네일 링크 수집
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.bd-list-thumb-a a")
            )
        )
        for el in elements:
            video_page_urls.append(el.get_attribute("href"))  # 상세 페이지 링크 저장

        # ✅ 수집된 각 게시물에서 동영상 추출 및 다운로드
        for idx, post_url in enumerate(video_page_urls, 1):
            driver.get(post_url)  # 게시물 상세 페이지로 이동

            # 게시물 제목 추출
            title = (
                WebDriverWait(driver, 10)
                .until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.name")))
                .text
            )

            # <video> 태그에서 직접 동영상 파일 주소(src) 추출
            video = driver.find_element(By.TAG_NAME, "video").get_attribute("src")

            # 파일명 안전하게 구성 (한글/특수문자 제거)
            filename = "".join(
                c for c in title if c.isalnum() or c in (" ", "_")
            ).replace(" ", "_")
            path = os.path.join(DOWNLOAD_DIR, f"{filename}.mp4")

            # ✅ 동영상 파일 다운로드 (스트리밍 방식)
            r = requests.get(video, stream=True, timeout=30)
            if r.status_code == 200:
                with open(path, "wb") as f:
                    for chunk in r.iter_content(8192):  # 8KB씩 나눠 저장
                        f.write(chunk)

    # ❗ 드라이버 예외가 발생하면 재시작
    except WebDriverException:
        driver.quit()
        driver = init_driver()

    # ❗ 기타 오류 출력
    except Exception as e:
        print(f"오류: {e}")

driver.quit()  # 마지막에 드라이버 종료